In [1]:
import pymongo
import time
import threading
import random

In [2]:
cl = pymongo.MongoClient("mongodb://localhost:27017")
db = cl["deep"]
obj1 = db["obj1"]
obj2 = db["obj2"]
obj4 = db["obj4"]
obj8 = db["obj8"]
nested_objs = [obj1, obj2, obj4, obj8]
users_list = [1, 2, 4]
depth_list = [0, 1, 2, 3]

In [3]:
obj1.find_one()

{'_id': ObjectId('68722a14a521c06a4f991eca'),
 'l1': {'device': 975, 'app': 'five', 'volume_total_bytes': 5367}}

In [4]:
obj1.count_documents({"l1.device": {"$gte": 5000}})

49747

In [5]:
obj1.count_documents({})

100000

In [6]:
filter_num=5000
l1_where_index = {"l1.device": {"$gte": filter_num}}
l2_where_index = {"l1.l2.device": {"$gte": filter_num}}
l4_where_index = {"l1.l2.l3.l4.device": {"$gte": filter_num}}
l8_where_index = {"l1.l2.l3.l4.l5.l6.l7.l8.device": {"$gte": filter_num}}
where_list = [l1_where_index, l2_where_index, l4_where_index, l8_where_index]

In [7]:
def create_l1(size):
    batch = []
    for _ in range(size):
        el = {
            "l1": {
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "volume_total_bytes": random.randint(0,9999)
            }
        }
        batch.append(el)

    return batch

In [8]:
def create_l2(size):
    batch = []
    for _ in range(size):
        el = {
            "l1":{
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "l2": {
                    "device": random.randint(5000,9999),
                    "app": "deepbench",
                    "volume_total_bytes": random.randint(0,9999)
                }
            }
        }
        batch.append(el)

    return batch

In [9]:
def create_l4(size):
    batch = []
    for _ in range(size):
        el = {
            "l1":{
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "l2": {
                    "device": random.randint(5000,9999),
                    "app": "deepbench",
                    "l3": {
                        "device": random.randint(5000,9999),
                        "app": "deepbench",
                        "l4": {
                            "device": random.randint(5000,9999),
                            "app": "deepbench",
                            "volume_total_bytes": random.randint(0,9999)
                        }
                    }
                }
            }
        }
        batch.append(el)

    return batch

In [10]:
def create_l8(size):
    batch = []
    for _ in range(size):
        el = {
            "l1":{
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "l2": {
                    "device": random.randint(5000,9999),
                    "app": "deepbench",
                    "l3": {
                        "device": random.randint(5000,9999),
                        "app": "deepbench",
                        "l4": {
                            "device": random.randint(5000,9999),
                            "app": "deepbench",
                            "l5": {
                                "device": random.randint(5000,9999),
                                "app": "deepbench",
                                "l6": {
                                    "device": random.randint(5000,9999),
                                    "app": "deepbench",
                                    "l7": {
                                        "device": random.randint(5000,9999),
                                        "app": "deepbench",
                                        "l8": {
                                            "device": random.randint(5000,9999),
                                            "app": "deepbench",
                                            "volume_total_bytes": random.randint(0,9999),
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        batch.append(el)

    return batch

In [11]:
def createIndexes():
    obj1.create_index([("l1.device", 1)])
    obj2.create_index([("l1.l2.device", 1)])
    obj4.create_index([("l1.l2.l3.l4.device", 1)])
    obj8.create_index([("l1.l2.l3.l4.l5.l6.l7.l8.device", 1)])

In [12]:
def clearIndexes():
    obj1.drop_indexes()
    obj2.drop_indexes()
    obj4.drop_indexes()
    obj8.drop_indexes()

In [13]:
def showIndexes():
    for i in obj1.list_indexes():
        if i["name"] != "_id_":
            print(i)
    for a in obj2.list_indexes():
        if i["name"] != "_id_":
            print(a)
    for b in obj4.list_indexes():
        if i["name"] != "_id_":
            print(b)
    for c in obj8.list_indexes():
        if i["name"] != "_id_":
            print(c)

In [14]:
def undo_old():
    unset_pipe_1 = [
        {"$set": {"l1.device": "$l1.old_device"}},
        {"$unset": "l1.old_device"}
    ]
    unset_pipe_2 = [
        {"$set": {"l1.l2.device": "$l1.l2.old_device"}},
        {"$unset": "l1.l2.old_device"},
    ]
    unset_pipe_4 = [
        {"$set": {"l1.l2.l3.l4.device": "$l1.l2.l3.l4.old_device"}},
        {"$unset": "l1.l2.l3.l4.old_device"},
    ]
    unset_pipe_8 = [
        {"$set": {"l1.l2.l3.l4.l5.l6.l7.l8.device": "$l1.l2.l3.l4.l5.l6.l7.l8.old_device"}},
        {"$unset": "l1.l2.l3.l4.l5.l6.l7.l8.old_device"}
    ]
    
    obj1.update_many({"l1.old_device": {"$exists": True}}, unset_pipe_1)
    obj2.update_many({"l1.l2.old_device": {"$exists": True}}, unset_pipe_2)
    obj4.update_many({"l1.l2.l3.l4.old_device": {"$exists": True}},unset_pipe_4)
    obj8.update_many({"l1.l2.l3.l4.l5.l6.l7.l8.old_device": {"$exists": True}},unset_pipe_8)

In [15]:
def insert_mocks():
    batch_1 = create_l1(10000)
    batch_2 = create_l2(10000)
    batch_4 = create_l4(10000)
    batch_8 = create_l8(10000)
    obj1.insert_many(batch_1)
    obj2.insert_many(batch_2)
    obj4.insert_many(batch_4)
    obj8.insert_many(batch_8)

In [16]:
def delete_mocks():
    obj1.delete_many({"l1.device": {"$gte": 5000}})
    obj2.delete_many({"l1.l2.device": {"$gte": 5000}})
    obj4.delete_many({"l1.l2.l3.l4.device": {"$gte": 5000}})
    obj8.delete_many({"l1.l2.l3.l4.l5.l6.l7.l8.device": {"$gte": 5000}})

In [17]:
def prepare():
    update_pipe_1 = [
        {"$set": {"l1.old_device": "$l1.device"}},
        {"$set": {"l1.device": 0}},

    ]
    update_pipe_2 = [
        {"$set": {"l1.l2.old_device": "$l1.l2.device"}},
        {"$set": {"l1.l2.device": 0}},
    ]
    update_pipe_4 = [
        {"$set": {"l1.l2.l3.l4.old_device": "$l1.l2.l3.l4.device"}},
        {"$set": {"l1.l2.l3.l4.device": 0}},
    ]
    update_pipe_8 = [
        {"$set": {"l1.l2.l3.l4.l5.l6.l7.l8.old_device": "$l1.l2.l3.l4.l5.l6.l7.l8.device"}},
        {"$set": {"l1.l2.l3.l4.l5.l6.l7.l8.device": 0}},
    ]
    obj1.update_many(l1_where_index, update_pipe_1)
    obj2.update_many(l2_where_index, update_pipe_2)
    obj4.update_many(l4_where_index, update_pipe_4)
    obj8.update_many(l8_where_index, update_pipe_8)

In [18]:
def undo_prepare():
    delete_mocks()
    undo_old()

In [19]:
def create_l1(size):
    batch = []
    for _ in range(size):
        el = {
            "l1": {
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "volume_total_bytes": random.randint(0,9999)
            }
        }
        batch.append(el)

    return batch

In [20]:
def create_l2(size):
    batch = []
    for _ in range(size):
        el = {
            "l1":{
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "l2": {
                    "device": random.randint(5000,9999),
                    "app": "deepbench",
                    "volume_total_bytes": random.randint(0,9999)
                }
            }
        }
        batch.append(el)

    return batch

In [21]:
def create_l4(size):
    batch = []
    for _ in range(size):
        el = {
            "l1":{
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "l2": {
                    "device": random.randint(5000,9999),
                    "app": "deepbench",
                    "l3": {
                        "device": random.randint(5000,9999),
                        "app": "deepbench",
                        "l4": {
                            "device": random.randint(5000,9999),
                            "app": "deepbench",
                            "volume_total_bytes": random.randint(0,9999)
                        }
                    }
                }
            }
        }
        batch.append(el)

    return batch

In [22]:
def create_l8(size):
    batch = []
    for _ in range(size):
        el = {
            "l1":{
                "device": random.randint(5000,9999),
                "app": "deepbench",
                "l2": {
                    "device": random.randint(5000,9999),
                    "app": "deepbench",
                    "l3": {
                        "device": random.randint(5000,9999),
                        "app": "deepbench",
                        "l4": {
                            "device": random.randint(5000,9999),
                            "app": "deepbench",
                            "l5": {
                                "device": random.randint(5000,9999),
                                "app": "deepbench",
                                "l6": {
                                    "device": random.randint(5000,9999),
                                    "app": "deepbench",
                                    "l7": {
                                        "device": random.randint(5000,9999),
                                        "app": "deepbench",
                                        "l8": {
                                            "device": random.randint(5000,9999),
                                            "app": "deepbench",
                                            "volume_total_bytes": random.randint(0,9999),
                                        }
                                    }
                                }
                            }
                        }
                    }
                }
            }
        }
        batch.append(el)

    return batch

In [23]:
def measure_delete(user_id, duration_list, depth):
    duration = 0
    col = nested_objs[depth]
    where = where_list[depth]
    num_of_tries = 1
    
    for _ in range(num_of_tries):
        insert_mocks()
        start = time.perf_counter()
        col.delete_many(where)
        end = time.perf_counter()
        duration += end - start

    duration = (duration/num_of_tries)*1000
    duration_list[user_id] = duration

In [24]:
def test_delete(num_of_users, depth):
    threads_list = []
    duration_list = [None] * num_of_users

    for x in range(num_of_users):
        t = threading.Thread(target=measure_delete, args=(x, duration_list, depth,))
        t.start()
        threads_list.append(t)

    
    for t in threads_list:
        t.join()
        
    result = sum(duration_list) / len(duration_list)
    return result    

In [25]:
def execute_delete():
    u_len = len(users_list)
    d_len = len(depth_list)
    result = [[0 for _ in range(d_len)] for _ in range(u_len)]
    
    for u in range(u_len):
        for d in range(d_len):
            result[u][d] = test_delete(users_list[u], depth_list[d])

    undo_prepare()
    return result

In [26]:
def calculate_diffs(old, new):
    for u in range(len(users_list)):
        print(f"user {users_list[u]}")
        for d in range(len(depth_list)):
            o_val = old[u][d]
            n_val = new[u][d]
            diff = ((n_val - o_val) / o_val) * 100
            print(f"depth: {depth_list[d]}, {o_val:10.2f} {n_val:10.2f} {diff:10.2f}")
        print("--------")

In [27]:
showIndexes()

In [28]:
prepare()

In [29]:
print(obj1.count_documents(l1_where_index))
print(obj2.count_documents(l2_where_index))
print(obj4.count_documents(l4_where_index))
print(obj8.count_documents(l8_where_index))

0
0
0
0


In [31]:
no_index = execute_delete()

In [32]:
createIndexes()

In [33]:
showIndexes()

SON([('v', 2), ('key', SON([('l1.device', 1)])), ('name', 'l1.device_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('l1.l2.device', 1)])), ('name', 'l1.l2.device_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('l1.l2.l3.l4.device', 1)])), ('name', 'l1.l2.l3.l4.device_1')])
SON([('v', 2), ('key', SON([('_id', 1)])), ('name', '_id_')])
SON([('v', 2), ('key', SON([('l1.l2.l3.l4.l5.l6.l7.l8.device', 1)])), ('name', 'l1.l2.l3.l4.l5.l6.l7.l8.device_1')])


In [34]:
with_index = execute_delete()

In [35]:
calculate_diffs(no_index,with_index)

user 1
depth: 0,     126.25     867.89     587.42
depth: 1,     204.61     919.65     349.47
depth: 2,     280.17    1043.74     272.54
depth: 3,     359.43    1174.77     226.84
--------
user 2
depth: 0,     423.01     735.93      73.98
depth: 1,     517.97     766.94      48.07
depth: 2,     624.74     967.19      54.82
depth: 3,     690.49    1217.76      76.36
--------
user 4
depth: 0,    1295.16    1975.69      52.54
depth: 1,    1677.21    2551.19      52.11
depth: 2,    1766.35    2724.65      54.25
depth: 3,    2192.76    3439.34      56.85
--------
